In [2]:
# librairies

import xarray as xr
import numpy as np
import pandas as pd
import glob
from datetime import datetime
import gc
import os
import sys

# Add the path to the custom library
custom_library_path = os.path.abspath('/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/fabienVED')
sys.path.append(custom_library_path)
# set the directory path
os.chdir('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/')


import data_process
from tqdm import tqdm
import matplotlib.pyplot as plt

# Loop

In [3]:
# paths
#upath = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/10m_u_component_of_wind/'
#vpath = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/10m_v_component_of_wind/'
ifg = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/instantaneous_10m_wind_gust/'

# Date specification
year = np.arange(1990,2021,1)
#target_month = 2
#target_day = 1

In [2]:
# Gather all the starting days of the 96 storms from the time series datasets

# Load the time series datasets

dates = pd.read_csv('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/data/time_series_1h_EU/instantaneous_10m_wind_gust/instantaneous_10m_wind_gust_max.csv')
dates = dates['start_date']
eu_final_raster = xr.open_dataset('pre_processing/maps/eu_final_raster.tif',
                                    engine='rasterio')

# rename x and y to lon and lat
eu_final_raster = eu_final_raster.rename({'x':'longitude','y':'latitude'})

# Extract the starting days of the 96 storms

storm_dates = []
for date in dates:
    storm_dates.append(datetime.strptime(date, '%Y-%m-%dT%H:%M:%S'))

# Extract each month and day of the storm starting days

storm_year = []
storm_month = []
storm_day = []
for date in storm_dates:
    storm_year.append(date.year)
    storm_month.append(date.month)
    storm_day.append(date.day)

# Combine the month and day of the storm starting days into a single list

storm_year_month_day = []
storm_month_day = []
for i in range(len(storm_month)):
    storm_year_month_day.append([storm_year[i],storm_month[i], storm_day[i]])
    storm_month_day.append([storm_month[i], storm_day[i]])

# keep the only the days that don't repeat themselves
storm_month_day = np.array(storm_month_day)
storm_month_day = np.unique(storm_month_day, axis=0)

storm_year_month_day = np.array(storm_year_month_day)
storm_year_month_day = np.unique(storm_year_month_day, axis=0)


In [22]:
max_winds_europe = []
for target_month, target_day in tqdm(storm_month_day):
    # Filter Test storms
    if ((target_month==2) & (target_day==19)):
        yearin = np.setdiff1d(year,np.asarray([1997]))
    elif ((target_month==12) & (target_day==15)):
        yearin = np.setdiff1d(year,np.asarray([1999]))
    elif ((target_month==2) & (target_day==25)):
        yearin = np.setdiff1d(year,np.asarray([2002]))
    elif ((target_month==2) & (target_day==3)):
        yearin = np.setdiff1d(year,np.asarray([2011]))
    elif ((target_month==2) & (target_day==1)):
        yearin = np.setdiff1d(year,np.asarray([2016]))
    elif ((target_month==2) & (target_day==8)):
        yearin = np.setdiff1d(year,np.asarray([2016]))
    elif ((target_month==2) & (target_day==16)):
        yearin = np.setdiff1d(year,np.asarray([2020]))
    elif ((target_month==2) & (target_day==28)):
        yearin = np.setdiff1d(year,np.asarray([1990]))
    else:
        yearin = year.copy()
        
    # Process the remaining years
    for yearz in tqdm(yearin):
        # import data
        #u10path = upath+f'ERA5_{str(yearz)}-{str(target_month)}_10m_u_component_of_wind.nc'
        #v10path = vpath+f'ERA5_{str(yearz)}-{str(target_month)}_10m_v_component_of_wind.nc'
        i10fgpath = ifg+f'ERA5_{str(yearz)}-{str(target_month)}_instantaneous_10m_wind_gust.nc'
        
        # read files with xarray
        #u10 = xr.open_dataset(glob.glob(u10path)[0])
        #v10 = xr.open_dataset(glob.glob(v10path)[0])
        i10fg = xr.open_dataset(glob.glob(i10fgpath)[0])

        if target_month == 2 and target_day == 29:
            if yearz % 4 != 0:
                continue
            elif yearz % 100 == 0 and yearz % 400 != 0:
                continue

        # parse date
        first_true_index,last_true_index = data_process.parse_date_and_output_list(i10fg,target_month,target_day)

        # Convert longitude from 0-360 to -180 to 180
        #u10['longitude'] = ((u10['longitude'] + 180) % 360) - 180
        #v10['longitude'] = ((v10['longitude'] + 180) % 360) - 180
        i10fg['longitude'] = ((i10fg['longitude'] + 180) % 360) - 180
        # Sort the dataset along the new longitude axis
        #u10 = u10.sortby('longitude')
        #v10 = v10.sortby('longitude')
        i10fg = i10fg.sortby('longitude')

        # Get the Europe subregion
        i10fg_europe = i10fg.sel(latitude=slice(71, 30), longitude=slice(-15, 40))

        # parse date
        i10fg_europe_date = i10fg_europe.isel(time=slice(first_true_index, last_true_index + 1))

        # get wind speed
        del i10fg, i10fg_europe
        gc.collect()

        # get maximum wind for 1 year
        #max_wind_europe = wspd.max(dim='time')
        max_wind_europe = i10fg_europe_date.max(dim='time')

        max_winds_europe.append(max_wind_europe)
    
    # Save the maximum wind speed for day target_day of month target_month
    locals()['max_winds_europe_'+str(target_month)+'_'+str(target_day)] = xr.concat(max_winds_europe, dim="time").max('time')
    # Convert the dataset to a DataArray (if needed) and set CRS
    data_array = locals()['max_winds_europe_'+str(target_month)+'_'+str(target_day)]
    #data_array = data_array.rio.write_crs("EPSG:4326")  # Assign CRS (WGS84)
    test_cut = data_array.where(eu_final_raster['band_data'] == 1)
    dataset_cut = test_cut
    dataset_cut = dataset_cut.rio.write_crs("EPSG:4326")  # Assign CRS (WGS84)
    #output_path = f'data/climatology/instantaneous_10m_wind_gust_cut/{file[:-4]}_cut.tif'
    # drop spatial_ref attribute
    dataset_cut = dataset_cut.drop_vars('spatial_ref')
    # convert to 2D
    dataset_cut = dataset_cut.squeeze()
    # Save as a raster file (GeoTIFF)
    output_path = f"data/climatology/daily_with_storms/{'climatology_europe_'+str(target_month)+'_'+str(target_day)}.tif"
    dataset_cut.rio.to_raster(output_path)

 19%|█████████▏                                       | 11/59 [08:14<35:59, 44.99s/it]


KeyboardInterrupt: 

In [ ]:
#xr.concat(max_winds_europe, dim="time").mean('time').plot()
#test['i10fg'].plot()
for target_month, target_day in storm_month_day:
    ds = locals()['max_winds_europe_'+str(target_month)+'_'+str(target_day)]
    # Convert the dataset to a DataArray (if needed) and set CRS
    data_array = ds['i10fg']
    data_array = data_array.rio.write_crs("EPSG:4326")  # Assign CRS (WGS84)

    # Save as a raster file (GeoTIFF)
    output_path = f"data/climatology/daily_without_storms/{'climatology_europe_'+str(target_month)+'_'+str(target_day)}.tif"
    data_array.rio.to_raster(output_path)
'''
#plt.title('Daily Max Wind Climatology: 01.02')
#plt.savefig('./climatology.png',dpi=400)'''

"\n#plt.title('Daily Max Wind Climatology: 01.02')\n#plt.savefig('./climatology.png',dpi=400)"

# part for daily max WITH storms included (except test set)

In [6]:
import pandas as pd
import xarray as xr
import numpy as np
from datetime import datetime
from tqdm import tqdm
import gc

# Load time series dataset
dates = pd.read_csv('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/data/time_series_1h_EU/instantaneous_10m_wind_gust/instantaneous_10m_wind_gust_max.csv')['start_date']

# Load and preprocess the raster dataset
eu_final_raster = xr.open_dataset('pre_processing/maps/eu_final_raster.tif', engine='rasterio').rename({'x': 'longitude', 'y': 'latitude'})

# Parse storm dates
storm_dates = [datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in dates]

# Extract unique month-day combinations
storm_month_day = np.unique([[date.month, date.day] for date in storm_dates], axis=0)

# Define special exclusions for years based on month and day
exclusions = {
    (2, 19): [1997],
    (12, 15): [1999],
    (2, 25): [2002],
    (2, 3): [2011],
    (2, 1): [2016],
    (2, 8): [2016],
    (2, 16): [2020],
    (2, 28): [1990],
}

# Process each unique storm day
for target_month, target_day in tqdm(storm_month_day):
    yearin = np.setdiff1d(year, exclusions.get((target_month, target_day), []))

    max_winds_europe = []
    for yearz in tqdm(yearin):
        # Skip non-leap years for Feb 29
        if target_month == 2 and target_day == 29 and (yearz % 4 != 0 or (yearz % 100 == 0 and yearz % 400 != 0)):
            continue

        # Load dataset
        i10fgpath = f'{ifg}ERA5_{yearz}-{target_month}_instantaneous_10m_wind_gust.nc'
        i10fg = xr.open_dataset(glob.glob(i10fgpath)[0])

        # Parse date indices
        first_true_index, last_true_index = data_process.parse_date_and_output_list(i10fg, target_month, target_day)

        # Preprocess dataset
        i10fg['longitude'] = ((i10fg['longitude'] + 180) % 360) - 180
        i10fg = i10fg.sortby('longitude')
        i10fg_europe = i10fg.sel(latitude=slice(71, 30), longitude=slice(-15, 40))
        i10fg_europe_date = i10fg_europe.isel(time=slice(first_true_index, last_true_index + 1))

        # Calculate maximum wind speed
        max_wind_europe = i10fg_europe_date.max(dim='time')
        max_winds_europe.append(max_wind_europe)

        del i10fg, i10fg_europe
        gc.collect()

    # Combine and save results
    combined_max = xr.concat(max_winds_europe, dim="time").max('time')
    dataset_cut = combined_max.where(eu_final_raster['band_data'] == 1).rio.write_crs("EPSG:4326").squeeze().drop_vars('spatial_ref')

    output_path = f"data/climatology/daily_with_storms/climatology_europe_{target_month}_{target_day}.tif"
    dataset_cut.rio.to_raster(output_path)

  0%|                                                          | 0/59 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████| 59/59 [42:28<00:00, 43.20s/it]


# Part for daily max WITHOUT storms included, with 1 day before and after done

In [4]:
import pandas as pd
import xarray as xr
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import gc

# Load time series dataset
dates = pd.read_csv('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/data/time_series_1h_EU/instantaneous_10m_wind_gust/instantaneous_10m_wind_gust_max.csv')['start_date']

# Load and preprocess the raster dataset
eu_final_raster = xr.open_dataset('pre_processing/maps/eu_final_raster.tif', engine='rasterio').rename({'x': 'longitude', 'y': 'latitude'})

# Parse storm dates
storm_dates = [datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in dates]

# Extract unique month-day combinations with landfall years
storm_month_day_year = [[date.month, date.day, date.year] for date in storm_dates]
storm_month_day = np.unique([[mdy[0], mdy[1]] for mdy in storm_month_day_year], axis=0)

# Define special exclusions for specific years based on month and day
exclusions = {
    (2, 19): [1997],
    (12, 15): [1999],
    (2, 25): [2002],
    (2, 3): [2011],
    (2, 1): [2016],
    (2, 8): [2016],
    (2, 16): [2020],
    (2, 28): [1990],
}

# Helper function to add wrap-around logic for days
def get_extended_days(target_month, target_day):
    base_date = datetime(2000, target_month, target_day)  # Use 2000 as a dummy year for simplicity
    extended_days = [(base_date - timedelta(days=1)).month, (base_date - timedelta(days=1)).day], \
                    [base_date.month, base_date.day], \
                    [(base_date + timedelta(days=1)).month, (base_date + timedelta(days=1)).day]
    return extended_days

# Process each unique storm day
for target_month, target_day in tqdm(storm_month_day):
    # Get the extended range of days
    extended_days = get_extended_days(target_month, target_day)

    # Collect landfall years for all days in the extended range
    landfall_years = []
    for month, day in extended_days:
        landfall_years += [mdy[2] for mdy in storm_month_day_year if mdy[0] == month and mdy[1] == day]

    # Exclude specific years
    excluded_years = []
    for month, day in extended_days:
        excluded_years += exclusions.get((month, day), [])
    excluded_years += landfall_years
    yearin = np.setdiff1d(year, excluded_years)

    max_winds_europe = []
    for yearz in tqdm(yearin):
        for month, day in extended_days:
            # Skip non-leap years for Feb 29
            if month == 2 and day == 29 and (yearz % 4 != 0 or (yearz % 100 == 0 and yearz % 400 != 0)):
                continue

            # Load dataset
            i10fgpath = f'{ifg}ERA5_{yearz}-{month}_instantaneous_10m_wind_gust.nc'
            i10fg = xr.open_dataset(glob.glob(i10fgpath)[0])

            # Parse date indices
            first_true_index, last_true_index = data_process.parse_date_and_output_list(i10fg, month, day)

            # Preprocess dataset
            i10fg['longitude'] = ((i10fg['longitude'] + 180) % 360) - 180
            i10fg = i10fg.sortby('longitude')
            i10fg_europe = i10fg.sel(latitude=slice(71, 30), longitude=slice(-15, 40))
            i10fg_europe_date = i10fg_europe.isel(time=slice(first_true_index, last_true_index + 1))

            # Calculate maximum wind speed
            max_wind_europe = i10fg_europe_date.max(dim='time')
            max_winds_europe.append(max_wind_europe)

            del i10fg, i10fg_europe
            gc.collect()

    # Combine and save results
    combined_max = xr.concat(max_winds_europe, dim="time").max('time')
    dataset_cut = combined_max.where(eu_final_raster['band_data'] == 1).rio.write_crs("EPSG:4326").squeeze().drop_vars('spatial_ref')

    #output_path = f"data/climatology/daily_without_storms/climatology_europe_{target_month}_{target_day}.tif"
    output_path = f"test/climatology_europe_{target_month}_{target_day}.tif"
    dataset_cut.rio.to_raster(output_path)

  0%|                                                                                                                                  | 0/59 [00:00<?, ?it/s]

 39%|██████████████████████████████████████████████                                                                        | 23/59 [44:24<1:09:30, 115.84s/it]


KeyboardInterrupt: 

# part for daily max WITHOUT storms included

In [5]:
import pandas as pd
import xarray as xr
import numpy as np
from datetime import datetime
from tqdm import tqdm
import gc

# Load time series dataset
dates = pd.read_csv('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/data/time_series_1h_EU/instantaneous_10m_wind_gust/instantaneous_10m_wind_gust_max.csv')['start_date']

# Load and preprocess the raster dataset
eu_final_raster = xr.open_dataset('pre_processing/maps/eu_final_raster.tif', engine='rasterio').rename({'x': 'longitude', 'y': 'latitude'})

# Parse storm dates
storm_dates = [datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in dates]

# Extract unique month-day combinations with landfall years
storm_month_day_year = [[date.month, date.day, date.year] for date in storm_dates]
storm_month_day = np.unique([[mdy[0], mdy[1]] for mdy in storm_month_day_year], axis=0)

# Define special exclusions for specific years based on month and day
exclusions = {
    (2, 19): [1997],
    (12, 15): [1999],
    (2, 25): [2002],
    (2, 3): [2011],
    (2, 1): [2016],
    (2, 8): [2016],
    (2, 16): [2020],
    (2, 28): [1990],
}

# Process each unique storm day
for target_month, target_day in tqdm(storm_month_day):
    # Identify landfall year(s) for the current month-day combination
    landfall_years = [mdy[2] for mdy in storm_month_day_year if mdy[0] == target_month and mdy[1] == target_day]
    excluded_years = exclusions.get((target_month, target_day), []) + landfall_years
    yearin = np.setdiff1d(year, excluded_years)

    max_winds_europe = []
    for yearz in tqdm(yearin):
        # Skip non-leap years for Feb 29
        if target_month == 2 and target_day == 29 and (yearz % 4 != 0 or (yearz % 100 == 0 and yearz % 400 != 0)):
            continue

        # Load dataset
        i10fgpath = f'{ifg}ERA5_{yearz}-{target_month}_instantaneous_10m_wind_gust.nc'
        i10fg = xr.open_dataset(glob.glob(i10fgpath)[0])

        # Parse date indices
        first_true_index, last_true_index = data_process.parse_date_and_output_list(i10fg, target_month, target_day)

        # Preprocess dataset
        i10fg['longitude'] = ((i10fg['longitude'] + 180) % 360) - 180
        i10fg = i10fg.sortby('longitude')
        i10fg_europe = i10fg.sel(latitude=slice(71, 30), longitude=slice(-15, 40))
        i10fg_europe_date = i10fg_europe.isel(time=slice(first_true_index, last_true_index + 1))

        # Calculate maximum wind speed
        max_wind_europe = i10fg_europe_date.max(dim='time')
        max_winds_europe.append(max_wind_europe)

        del i10fg, i10fg_europe
        gc.collect()

    # Combine and save results
    combined_max = xr.concat(max_winds_europe, dim="time").max('time')
    dataset_cut = combined_max.where(eu_final_raster['band_data'] == 1).rio.write_crs("EPSG:4326").squeeze().drop_vars('spatial_ref')

    output_path = f"data/climatology/daily_without_storms/climatology_europe_{target_month}_{target_day}.tif"
    dataset_cut.rio.to_raster(output_path)

  0%|                                                          | 0/59 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████| 59/59 [41:04<00:00, 41.78s/it]


In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
from datetime import datetime
from tqdm import tqdm
import gc

# Load time series dataset
dates = pd.read_csv('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/data/time_series_1h_EU/instantaneous_10m_wind_gust/instantaneous_10m_wind_gust_max.csv')['start_date']

# Load and preprocess the raster dataset
eu_final_raster = xr.open_dataset('pre_processing/maps/eu_final_raster.tif', engine='rasterio').rename({'x': 'longitude', 'y': 'latitude'})

# Parse storm dates
storm_dates = [datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in dates]

# Extract unique month-day combinations with landfall years
storm_month_day_year = [[date.month, date.day, date.year] for date in storm_dates]
storm_month_day = np.unique([[mdy[0], mdy[1]] for mdy in storm_month_day_year], axis=0)

# Define special exclusions for specific years based on month and day
exclusions = {
    (2, 19): [1997],
    (12, 15): [1999],
    (2, 25): [2002],
    (2, 3): [2011],
    (2, 1): [2016],
    (2, 8): [2016],
    (2, 16): [2020],
    (2, 28): [1990],
}

# Process each unique storm day
for target_month, target_day in tqdm(storm_month_day):
    # Identify landfall year(s) for the current month-day combination
    landfall_years = [mdy[2] for mdy in storm_month_day_year if mdy[0] == target_month and mdy[1] == target_day]
    excluded_years = exclusions.get((target_month, target_day), []) + landfall_years
    yearin = np.setdiff1d(year, excluded_years)

    max_winds_europe = []
    for yearz in tqdm(yearin):
        # Skip non-leap years for Feb 29
        if target_month == 2 and target_day == 29 and (yearz % 4 != 0 or (yearz % 100 == 0 and yearz % 400 != 0)):
            continue

        # Load dataset
        i10fgpath = f'{ifg}ERA5_{yearz}-{target_month}_instantaneous_10m_wind_gust.nc'
        i10fg = xr.open_dataset(glob.glob(i10fgpath)[0])

        # Parse date indices
        first_true_index, last_true_index = data_process.parse_date_and_output_list(i10fg, target_month, target_day)

        # Preprocess dataset
        i10fg['longitude'] = ((i10fg['longitude'] + 180) % 360) - 180
        i10fg = i10fg.sortby('longitude')
        i10fg_europe = i10fg.sel(latitude=slice(71, 30), longitude=slice(-15, 40))
        i10fg_europe_date = i10fg_europe.isel(time=slice(first_true_index, last_true_index + 1))

        # Calculate maximum wind speed
        max_wind_europe = i10fg_europe_date.max(dim='time')
        max_winds_europe.append(max_wind_europe)

        del i10fg, i10fg_europe
        gc.collect()

    # Combine and save results
    combined_max = xr.concat(max_winds_europe, dim="time").max('time')
    dataset_cut = combined_max.where(eu_final_raster['band_data'] == 1).rio.write_crs("EPSG:4326").squeeze().drop_vars('spatial_ref')

    output_path = f"data/climatology/daily_without_storms/climatology_europe_{target_month}_{target_day}.tif"
    dataset_cut.rio.to_raster(output_path)